<h1>Stacked Auto Encoders Training 7-13-2016 With Noise</h1>

Assuming CNN-H1, and the 6 CNNs for the patches have been trained, we can use the weights to get activations for their fc6 layer and train 3 layers of our SAE individually.

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%matplotlib inline

import theano.sandbox.cuda
theano.sandbox.cuda.use('gpu1')

import os
import numpy as np
import matplotlib.pyplot as plt

from multiprocessing import Process
from scipy.ndimage.filters import gaussian_filter

from skimage import io
from skimage.color import rgb2grey
from skimage.transform import resize

from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers

# from keras.utils import np_utils

np.random.seed(123456)

Using gpu device 1: GRID K520 (CNMeM is enabled with initial size: 98.0% of memory, CuDNN 3007)
Using Theano backend.


In [3]:
cnn_h1_weight_file = 'CNN-H1_epoch-20_activations'
cnn_p1_weight_file = 'CNN-P1_epoch-20_activations'
cnn_p2_weight_file = 'CNN-P2_epoch-20_activations'
cnn_p3_weight_file = 'CNN-P3_epoch-20_activations'
cnn_p4_weight_file = 'CNN-P4_epoch-20_activations'
cnn_p5_weight_file = 'CNN-P5_epoch-20_activations'
cnn_p6_weight_file = 'CNN-P6_epoch-20_activations'
y_train_file = 'y_train_activations'

layer_1_size = 2048
layer_2_size = 1024
layer_3_size = 512

validation_split = .15


In [70]:
def load_activations(file_name):
    return np.load('models/' + file_name + '.npy')

def save_activations(file_name, activations):
    return np.save('models/' + file_name + '.npy', activations)

In [5]:
cnn_h1_activations = load_activations(cnn_h1_weight_file)
cnn_p1_activations = load_activations(cnn_p1_weight_file)
cnn_p2_activations = load_activations(cnn_p2_weight_file)
cnn_p3_activations = load_activations(cnn_p3_weight_file)
cnn_p4_activations = load_activations(cnn_p4_weight_file)
cnn_p5_activations = load_activations(cnn_p5_weight_file)
cnn_p6_activations = load_activations(cnn_p6_weight_file)
y_train = load_activations(y_train_file)

In [6]:
def fix_activations(_activations):
    return np.array([activation for activations in _activations for activation in activations])

In [7]:
cnn_h1_activations = fix_activations(cnn_h1_activations)
cnn_p1_activations = fix_activations(cnn_p1_activations)
cnn_p2_activations = fix_activations(cnn_p2_activations)
cnn_p3_activations = fix_activations(cnn_p3_activations)
cnn_p4_activations = fix_activations(cnn_p4_activations)
cnn_p5_activations = fix_activations(cnn_p5_activations)
cnn_p6_activations = fix_activations(cnn_p6_activations)

In [8]:
cnn_h1_activations.shape

(26668, 512)

In [9]:
activations = np.concatenate((
        cnn_h1_activations, 
        cnn_p1_activations,
        cnn_p2_activations,
        cnn_p3_activations,
        cnn_p4_activations,
        cnn_p5_activations,
        cnn_p6_activations), axis=1)

In [10]:
activations.shape

(26668, 3584)

In [11]:
y_train.shape

(26668,)

<h2>Random Noise</h2>

In [59]:
def random_noise(activations, random_threshold, gaussian_scale):
    return np.array([np.array([value + np.random.normal(scale=gaussian_scale) 
                      if np.random.rand() < random_threshold else value
                      for value in activation]) for activation in activations])       

In [65]:
activations_with_noise_05_1 = random_noise(activations, .05, 1)

KeyboardInterrupt: 

In [71]:
save_activations('activations_with_noise_05_1', activations_with_noise_05_1)

<h2>Auto Encoder</h2>

In [66]:
def get_encoder(X_train, X_test, encoding_dim, nb_epoch=50, regularizer=10e-5):
    assert X_train.shape[1] == X_test.shape[1]
    shape = X_train.shape[1]

    input_img = Input(shape=(shape,))
    encoded = Dense(
        encoding_dim, 
        activation='relu', 
        activity_regularizer=regularizers.activity_l1(regularizer)
    )(input_img)
    decoded = Dense(shape, activation='sigmoid')(encoded)

    autoencoder = Model(input=input_img, output=decoded)
    encoder = Model(input=input_img, output=encoded)

    # sgd = SGD(lr=.01, decay=99.9) #lr=0.01->0.00001
    autoencoder.compile(
        optimizer='adadelta', 
        loss='categorical_crossentropy', 
        metrics=['accuracy'])
    
    autoencoder.fit(X_train, X_train,
                nb_epoch=nb_epoch,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))
    return autoencoder, encoder
    

In [67]:
activations_with_noise = load_activations('activations_with_noise_05_1')

x_train = activations_with_noise[:int(activations_with_noise.shape[0] * (1 - validation_split))]
x_test = activations_with_noise[int(activations_with_noise.shape[0] * (1 - validation_split)):]
X_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
X_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))

In [68]:
autoencoder1, encoder1 = get_encoder(X_train, X_test, 2048, 30, 10e-3)

Train on 22667 samples, validate on 4001 samples
Epoch 1/30
22667/22667 [==============================] - 4s - loss: -58107.9589 - acc: 0.0521 - val_loss: -65617.0481 - val_acc: 0.0932
Epoch 2/30
22667/22667 [==============================] - 4s - loss: -67561.0522 - acc: 0.1230 - val_loss: -68496.7222 - val_acc: 0.1400
Epoch 3/30
22667/22667 [==============================] - 4s - loss: -69732.6775 - acc: 0.1687 - val_loss: -70293.8552 - val_acc: 0.1930
Epoch 4/30
22667/22667 [==============================] - 4s - loss: -70999.9231 - acc: 0.1922 - val_loss: -70853.5306 - val_acc: 0.2002
Epoch 5/30
22667/22667 [==============================] - 4s - loss: -71825.0211 - acc: 0.2205 - val_loss: -71732.4067 - val_acc: 0.2287
Epoch 6/30
22667/22667 [==============================] - 4s - loss: -72462.4277 - acc: 0.2398 - val_loss: -72306.9324 - val_acc: 0.2467
Epoch 7/30
22667/22667 [==============================] - 4s - loss: -72936.4505 - acc: 0.2550 - val_loss: -72616.5206 - val_acc:

In [72]:
X_train = encoder1.predict(X_train)
X_test = encoder1.predict(X_test)

print X_train.shape
print X_test.shape

/usr/local/lib/python2.7/dist-packages/keras/backend/theano_backend.py:514: UserWarning: theano.function was asked to create a function computing outputs given certain inputs, but the provided input variable at index 1 is not part of the computational graph needed to compute the outputs: keras_learning_phase.
To make this warning into an error, you can pass the parameter on_unused_input='raise' to theano.function. To disable it completely, use on_unused_input='ignore'.
  **kwargs)


(22667, 2048)
(4001, 2048)


In [73]:
autoencoder2, encoder2 = get_encoder(X_train, X_test, 1024, 50)

Train on 22667 samples, validate on 4001 samples
Epoch 1/50
22667/22667 [==============================] - 2s - loss: 90442.2203 - acc: 3.5294e-04 - val_loss: 89547.5544 - val_acc: 2.4994e-04
Epoch 2/50
22667/22667 [==============================] - 2s - loss: 89405.9744 - acc: 0.0014 - val_loss: 88747.3123 - val_acc: 0.0027
Epoch 3/50
22667/22667 [==============================] - 2s - loss: 88353.1115 - acc: 0.0255 - val_loss: 87384.0254 - val_acc: 0.0712
Epoch 4/50
22667/22667 [==============================] - 2s - loss: 86846.8374 - acc: 0.1025 - val_loss: 86155.4334 - val_acc: 0.1310
Epoch 5/50
22667/22667 [==============================] - 2s - loss: 85641.2681 - acc: 0.1631 - val_loss: 85221.2403 - val_acc: 0.1900
Epoch 6/50
22667/22667 [==============================] - 2s - loss: 84722.3260 - acc: 0.2237 - val_loss: 84636.2191 - val_acc: 0.2402
Epoch 7/50
22667/22667 [==============================] - 2s - loss: 84071.4254 - acc: 0.2619 - val_loss: 84059.1067 - val_acc: 0.301

In [74]:
X_train = encoder2.predict(X_train)
X_test = encoder2.predict(X_test)

print X_train.shape
print X_test.shape

(22667, 1024)
(4001, 1024)


In [75]:
autoencoder3, encoder3 = get_encoder(X_train, X_test, 512, 50)

Train on 22667 samples, validate on 4001 samples
Epoch 1/50
22667/22667 [==============================] - 1s - loss: 13569.0686 - acc: 0.0015 - val_loss: 12781.3793 - val_acc: 0.0015
Epoch 2/50
22667/22667 [==============================] - 1s - loss: 12897.6966 - acc: 0.0101 - val_loss: 12230.6805 - val_acc: 0.0702
Epoch 3/50
22667/22667 [==============================] - 1s - loss: 12303.5779 - acc: 0.2299 - val_loss: 11746.7620 - val_acc: 0.3019
Epoch 4/50
22667/22667 [==============================] - 1s - loss: 11887.7001 - acc: 0.3334 - val_loss: 11448.5548 - val_acc: 0.3777
Epoch 5/50
22667/22667 [==============================] - 1s - loss: 11630.4836 - acc: 0.3827 - val_loss: 11278.6853 - val_acc: 0.4181
Epoch 6/50
22667/22667 [==============================] - 1s - loss: 11465.8468 - acc: 0.4077 - val_loss: 11167.9857 - val_acc: 0.3839
Epoch 7/50
22667/22667 [==============================] - 1s - loss: 11346.3691 - acc: 0.4156 - val_loss: 11091.4286 - val_acc: 0.4029
Epoch 

In [76]:
X_train = encoder3.predict(X_train)
X_test = encoder3.predict(X_test)

print X_train.shape
print X_test.shape

(22667, 512)
(4001, 512)


<h2>Train all together</h2>

In [14]:
input_img = Input(shape=(3584,))
encoded = Dense(2048, activation='relu')(input_img)
encoded = Dense(1024, activation='relu')(encoded)
encoded = Dense(512, activation='relu')(encoded)

decoded = Dense(1024, activation='relu')(encoded)
decoded = Dense(2048, activation='relu')(decoded)
decoded = Dense(3584, activation='sigmoid')(decoded)

autoencoder = Model(input=input_img, output=decoded)
autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])

autoencoder.fit(X_train, X_train,
                nb_epoch=100,
                batch_size=256,
                shuffle=True,
                validation_data=(X_test, X_test))

Train on 22667 samples, validate on 4001 samples
Epoch 1/100
22667/22667 [==============================] - 4s - loss: -30.7396 - acc: 0.0057 - val_loss: -38.3032 - val_acc: 0.0122
Epoch 2/100
22667/22667 [==============================] - 4s - loss: -40.9426 - acc: 0.0139 - val_loss: -42.7448 - val_acc: 0.0165
Epoch 3/100
22667/22667 [==============================] - 4s - loss: -45.7900 - acc: 0.0219 - val_loss: -47.3148 - val_acc: 0.0320
Epoch 4/100
22667/22667 [==============================] - 4s - loss: -48.7072 - acc: 0.0340 - val_loss: -49.0509 - val_acc: 0.0430
Epoch 5/100
22667/22667 [==============================] - 4s - loss: -50.5351 - acc: 0.0434 - val_loss: -51.4600 - val_acc: 0.0517
Epoch 6/100
22667/22667 [==============================] - 4s - loss: -52.1692 - acc: 0.0507 - val_loss: -52.2314 - val_acc: 0.0570
Epoch 7/100
22667/22667 [==============================] - 4s - loss: -53.5487 - acc: 0.0566 - val_loss: -53.6521 - val_acc: 0.0740
Epoch 8/100
22667/22667 [==